In [10]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import preprocessing

import datetime
from kerastuner.tuners import RandomSearch
from tensorflow.keras.backend import clear_session

from tensorboard.plugins.hparams import api as hp
from tensorflow import feature_column
from tensorflow.keras import layers

from sklearn.model_selection import KFold, StratifiedKFold
import math 

In [11]:
def process_data():
    df_test = pd.read_csv(r'test_data.csv', sep = ",", engine='python')
    df_train = pd.read_csv(r'train_data.csv', sep = ",", engine='python')

    test = df_test.drop(df_test.columns[0],axis=1)

    test_targets = test.iloc[:,[13]]
    test_inputs = test.loc[:,test.columns != 'target']

    train = df_train.drop(df_train.columns[0],axis=1)
    train_targets = train.iloc[:,[13]]
    train_inputs = train.loc[:,train.columns != 'target']
    
    np.savez('Spotify_data_train', inputs=train_inputs, targets=train_targets)
    np.savez('Spotify_data_test', inputs=test_inputs, targets=test_targets)

    npz = np.load('Spotify_data_train.npz')
    train_inputs,train_targets = npz['inputs'].astype(np.float),npz['targets'].astype(np.int)

    npz = np.load('Spotify_data_test.npz')
    test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

    test_inputs = preprocessing.scale(test_inputs)
    
    return train_inputs,train_targets,test_inputs,test_targets

train_inputs,train_targets,test_inputs,test_targets = process_data()
total_samples = len(train_inputs)
split = math.ceil(total_samples/5)
k = 5

In [12]:
def crossFold(train_inputs,train_targets,i):
    
    if(i == 4):
        validation_inputs = preprocessing.scale(train_inputs[i*split:])
        validation_targets = train_targets[i*split:]
        inputs = preprocessing.scale(train_inputs[:i*split])
        targets = train_targets[:i*split]
    elif(i==0):
        validation_inputs = preprocessing.scale(train_inputs[:split])
        validation_targets = train_targets[:split]
        inputs = preprocessing.scale(train_inputs[split:])
        targets = train_targets[split:]
    else:
        i1 = train_inputs[:i*split]
        i2 = train_inputs[(i+1)*split:]
        inputs = preprocessing.scale(np.concatenate((i1,i2)))
        
        it1 = train_targets[:i*split]
        it2 = train_targets[(i+1)*split:]
        targets = np.concatenate((it1,it2))
        
        validation_inputs = preprocessing.scale(train_inputs[i*split:(i+1)*split])
        validation_targets = train_targets[i*split:(i+1)*split]
        
    return inputs,targets,validation_inputs,validation_targets


In [13]:
#def run_session(num_layers,num_nodes,optimizers,reg_interval,batch_size,max_epochs):
    
HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([1,5])) 
HP_NUM_NODES = hp.HParam('num_nodes', hp.Discrete([4,8]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam','sgd']))
#['adam', 'sgd','RMSprop']

HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([210,300])) 
HP_MAX_EPOCHS = hp.HParam('max_epochs', hp.Discrete([12,25])) 

HP_L2 = hp.HParam('l2 regularizer', hp.RealInterval(0.001,0.01))
#.001,.01
METRIC_ACCURACY = 'accuracy'

METRICS = [
    hp.Metric(
        "val_accuracy", group="validation", display_name="accuracy (val.)",
    ),
    hp.Metric("val_loss", group="validation", display_name="loss (val.)",),
    hp.Metric(
        "accuracy", group="train", display_name="accuracy (train)",
    ),
    hp.Metric("loss", group="train", display_name="loss (train)",),
]
METRICS = np.asarray(METRICS)


with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
    hparams=[HP_NUM_LAYERS,HP_NUM_NODES,HP_OPTIMIZER,HP_BATCH_SIZE, HP_MAX_EPOCHS,HP_L2], 
    #metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    #metrics=METRICS,
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )
   
        
def train_test_model(run_dir,hparams,t_inputs,t_targets,v_inputs,v_targets): 
    
    
    input_size = 18 
    output_size = 2
    
    
    model = tf.keras.Sequential()
    model.add(layers.Dense(hparams[HP_NUM_NODES], activation='relu',input_shape=(18,)))
    for i in range(hparams[HP_NUM_LAYERS]-1):
        model.add(layers.Dense(hparams[HP_NUM_NODES],kernel_regularizer=tf.keras.regularizers.l2(0.001), activation='relu'))
    model.add(layers.Dense(output_size, activation='softmax'))
    
    #model.compile(optimizer=hparams[HP_OPTIMIZER], loss='sparse_categorical_crossentropy', metrics=['accuracy',get_f1])
    model.compile(optimizer=hparams[HP_OPTIMIZER], loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    #logdir = run_dir + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=path, histogram_freq=1)

    batch_size = hparams[HP_BATCH_SIZE]
    max_epochs = hparams[HP_MAX_EPOCHS]
    
        
    history = model.fit(  t_inputs,t_targets,
                          batch_size=batch_size,
                          epochs=max_epochs, 
                          validation_data=(v_inputs, v_targets),
                          callbacks=[
                                #tf.keras.callbacks.TensorBoard(run_dir,histogram_freq=1,update_freq='epoch'),  # log metrics
                                tf.keras.callbacks.TensorBoard(run_dir,histogram_freq=1,update_freq=1,embeddings_freq=1),
                                tf.keras.callbacks.EarlyStopping(
                                    monitor='val_accuracy', min_delta=0.001, patience=5, mode='max',
                                    baseline=None),
                                tf.keras.callbacks.EarlyStopping(
                                    monitor='val_loss', min_delta=0.001, patience=7, verbose=0, mode='min',
                                    baseline=0.6)

                                #hp.KerasCallback(run_dir, hparams),  # log hparams
                                #tf.keras.callbacks.TensorBoard.on_epoch_end(epoch,logs=None)
                                ],
                              verbose = 1) 

    test_loss,test_accuracy = model.evaluate(test_inputs, test_targets)
    
    #test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
    
    #train_loss = history.history['loss']
    #print(train_loss,'train\n')
    #val_loss = history.history['val_loss']
    #print(val_loss,'val \n')
    #tf.summary.scalar('runs_split{}'.format(session_num)+'Loss/train', train_loss)
    #tf.summary.scalar('runs_split{}'.format(session_num)+'Loss/val', val_loss)
    
    
    return test_loss, test_accuracy   
    
def run(run_dir,hparams,t_inputs,t_targets,v_inputs,v_targets):
    with tf.summary.create_file_writer(run_dir+"/Kfold"+str(i)).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(run_dir+"/Kfold"+str(i),hparams,t_inputs,t_targets,v_inputs,v_targets)
        #
        #accuracy= tf.reshape(tf.convert_to_tensor(accuracy), []).numpy()
        #
        tf.summary.scalar("loss",accuracy[0],step=session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy[1], step=session_num)

    return accuracy[1]
        

In [14]:
session_num = 0


for num_layers in HP_NUM_LAYERS.domain.values:
    for num_nodes in HP_NUM_NODES.domain.values:
        for optimizer in HP_OPTIMIZER.domain.values:
            for batch_size in HP_BATCH_SIZE.domain.values:
                for max_epoch in HP_MAX_EPOCHS.domain.values:
                        for l2 in (HP_L2.domain.min_value, HP_L2.domain.max_value):
                            hparams = {
                                HP_NUM_LAYERS: num_layers,
                                HP_NUM_NODES: num_nodes,
                                HP_OPTIMIZER: optimizer,
                                HP_BATCH_SIZE: batch_size, 
                                HP_MAX_EPOCHS: max_epoch,
                                HP_L2: l2,
                            }
                            combined_acc = 0
                            for i in range(5):
                                t_inputs,t_targets,v_inputs,v_targets = crossFold(train_inputs,train_targets,i)
                                run_name = "run-%d" % session_num
                                print('----------- Starting trial: %s' % run_name + '-----------')
                                print({h.name: hparams[h] for h in hparams})
                                print("Fold nr "+str(i+1))
                                
                                accuracy = run('logs/hparam_tuning/' + run_name, hparams,t_inputs,t_targets,v_inputs,v_targets)
                                combined_acc += accuracy
                            #accuracy = train_test_model(hparams)
                            #tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
                            combined_acc = combined_acc/k
                            tf.summary.scalar('Combined accuray', combined_acc , step=1)
                            print("Combined accuray: ",combined_acc)
                            session_num += 1

print("----- Session finished -----")


----------- Starting trial: run-0-----------
{'num_layers': 1, 'num_nodes': 4, 'optimizer': 'adam', 'batch_size': 210, 'max_epochs': 12, 'l2 regularizer': 0.001}
Fold nr 1
Epoch 1/12
47/47 - 1s - loss: 0.9196 - accuracy: 0.4398 - val_loss: 0.8605 - val_accuracy: 0.4329
Epoch 2/12
47/47 - 0s - loss: 0.8147 - accuracy: 0.4481 - val_loss: 0.7771 - val_accuracy: 0.4780
Epoch 3/12
47/47 - 0s - loss: 0.7563 - accuracy: 0.4775 - val_loss: 0.7310 - val_accuracy: 0.5158
Epoch 4/12
47/47 - 0s - loss: 0.7234 - accuracy: 0.5094 - val_loss: 0.7035 - val_accuracy: 0.5371
Epoch 5/12
47/47 - 0s - loss: 0.7023 - accuracy: 0.5291 - val_loss: 0.6852 - val_accuracy: 0.5589
Epoch 6/12
47/47 - 0s - loss: 0.6864 - accuracy: 0.5484 - val_loss: 0.6705 - val_accuracy: 0.5782
Epoch 7/12
47/47 - 0s - loss: 0.6731 - accuracy: 0.5709 - val_loss: 0.6582 - val_accuracy: 0.6020
96/96 [==============================] - ETA: 0s - loss: 0.6252 - accuracy: 0.56 - ETA: 0s - loss: 0.6574 - accuracy: 0.60 - ETA: 0s - loss: 0

96/96 [==============================] - ETA: 0s - loss: 0.5318 - accuracy: 0.75 - ETA: 0s - loss: 0.5340 - accuracy: 0.72 - 0s 935us/step - loss: 0.5367 - accuracy: 0.7334
Combined accuray:  0.6916612029075623
----------- Starting trial: run-1-----------
{'num_layers': 1, 'num_nodes': 4, 'optimizer': 'adam', 'batch_size': 210, 'max_epochs': 12, 'l2 regularizer': 0.01}
Fold nr 1
Epoch 1/12
47/47 - 1s - loss: 0.7779 - accuracy: 0.5349 - val_loss: 0.7557 - val_accuracy: 0.5384
Epoch 2/12
47/47 - 0s - loss: 0.7170 - accuracy: 0.5788 - val_loss: 0.7091 - val_accuracy: 0.5839
Epoch 3/12
47/47 - 1s - loss: 0.6827 - accuracy: 0.6172 - val_loss: 0.6799 - val_accuracy: 0.6200
Epoch 4/12
47/47 - 0s - loss: 0.6603 - accuracy: 0.6431 - val_loss: 0.6602 - val_accuracy: 0.6455
Epoch 5/12
47/47 - 0s - loss: 0.6440 - accuracy: 0.6622 - val_loss: 0.6455 - val_accuracy: 0.6606
Epoch 6/12
47/47 - 0s - loss: 0.6312 - accuracy: 0.6754 - val_loss: 0.6334 - val_accuracy: 0.6697
Epoch 7/12
47/47 - 0s - loss: 

Epoch 6/25
47/47 - 0s - loss: 0.6366 - accuracy: 0.6143 - val_loss: 0.6417 - val_accuracy: 0.6155
Epoch 7/25
47/47 - 0s - loss: 0.6204 - accuracy: 0.6576 - val_loss: 0.6254 - val_accuracy: 0.6668
96/96 [==============================] - ETA: 0s - loss: 0.5829 - accuracy: 0.68 - ETA: 0s - loss: 0.6155 - accuracy: 0.66 - 0s 756us/step - loss: 0.6172 - accuracy: 0.6648
----------- Starting trial: run-2-----------
{'num_layers': 1, 'num_nodes': 4, 'optimizer': 'adam', 'batch_size': 210, 'max_epochs': 25, 'l2 regularizer': 0.001}
Fold nr 2
Epoch 1/25
47/47 - 0s - loss: 1.1305 - accuracy: 0.4068 - val_loss: 1.0060 - val_accuracy: 0.4095
Epoch 2/25
47/47 - 0s - loss: 0.9511 - accuracy: 0.4147 - val_loss: 0.8648 - val_accuracy: 0.4218
Epoch 3/25
47/47 - 0s - loss: 0.8407 - accuracy: 0.4416 - val_loss: 0.7779 - val_accuracy: 0.4629
Epoch 4/25
47/47 - 0s - loss: 0.7708 - accuracy: 0.4760 - val_loss: 0.7257 - val_accuracy: 0.5043
Epoch 5/25
47/47 - 0s - loss: 0.7254 - accuracy: 0.5153 - val_loss:

Epoch 19/25
47/47 - 0s - loss: 0.5148 - accuracy: 0.7448 - val_loss: 0.5138 - val_accuracy: 0.7438
Epoch 20/25
47/47 - 0s - loss: 0.5137 - accuracy: 0.7439 - val_loss: 0.5132 - val_accuracy: 0.7438
Epoch 21/25
47/47 - 0s - loss: 0.5128 - accuracy: 0.7439 - val_loss: 0.5121 - val_accuracy: 0.7459
Epoch 22/25
47/47 - 0s - loss: 0.5121 - accuracy: 0.7434 - val_loss: 0.5115 - val_accuracy: 0.7463
Epoch 23/25
47/47 - 0s - loss: 0.5114 - accuracy: 0.7439 - val_loss: 0.5110 - val_accuracy: 0.7467
Epoch 24/25
47/47 - 0s - loss: 0.5108 - accuracy: 0.7445 - val_loss: 0.5105 - val_accuracy: 0.7471
Epoch 25/25
47/47 - 0s - loss: 0.5104 - accuracy: 0.7465 - val_loss: 0.5100 - val_accuracy: 0.7471
96/96 [==============================] - ETA: 0s - loss: 0.5078 - accuracy: 0.74 - ETA: 0s - loss: 0.5054 - accuracy: 0.74 - 0s 1ms/step - loss: 0.5050 - accuracy: 0.7482
Combined accuray:  0.685095202922821
----------- Starting trial: run-3-----------
{'num_layers': 1, 'num_nodes': 4, 'optimizer': 'adam',

Epoch 24/25
47/47 - 0s - loss: 0.5105 - accuracy: 0.7443 - val_loss: 0.5113 - val_accuracy: 0.7567
Epoch 25/25
47/47 - 0s - loss: 0.5099 - accuracy: 0.7457 - val_loss: 0.5107 - val_accuracy: 0.7571
96/96 [==============================] - ETA: 0s - loss: 0.4594 - accuracy: 0.71 - ETA: 0s - loss: 0.5011 - accuracy: 0.74 - 0s 756us/step - loss: 0.5079 - accuracy: 0.7446
----------- Starting trial: run-3-----------
{'num_layers': 1, 'num_nodes': 4, 'optimizer': 'adam', 'batch_size': 210, 'max_epochs': 25, 'l2 regularizer': 0.01}
Fold nr 4
Epoch 1/25
47/47 - 0s - loss: 0.8813 - accuracy: 0.4476 - val_loss: 0.8275 - val_accuracy: 0.4391
Epoch 2/25
47/47 - 0s - loss: 0.7817 - accuracy: 0.4704 - val_loss: 0.7523 - val_accuracy: 0.4764
Epoch 3/25
47/47 - 0s - loss: 0.7241 - accuracy: 0.5033 - val_loss: 0.7065 - val_accuracy: 0.5088
Epoch 4/25
47/47 - 0s - loss: 0.6881 - accuracy: 0.5355 - val_loss: 0.6744 - val_accuracy: 0.5544
Epoch 5/25
47/47 - 0s - loss: 0.6610 - accuracy: 0.5833 - val_loss

...

Epoch 4/25
47/47 - 0s - loss: 0.7134 - accuracy: 0.5101 - val_loss: 0.7136 - val_accuracy: 0.5166
Epoch 5/25
47/47 - 0s - loss: 0.7123 - accuracy: 0.5148 - val_loss: 0.7120 - val_accuracy: 0.5269
Epoch 6/25
47/47 - 0s - loss: 0.7111 - accuracy: 0.5265 - val_loss: 0.7107 - val_accuracy: 0.5396
Epoch 7/25
47/47 - 0s - loss: 0.7100 - accuracy: 0.5308 - val_loss: 0.7093 - val_accuracy: 0.5540
96/96 [==============================] - ETA: 0s - loss: 0.6840 - accuracy: 0.50 - ETA: 0s - loss: 0.7110 - accuracy: 0.53 - ETA: 0s - loss: 0.7100 - accuracy: 0.53 - 0s 1ms/step - loss: 0.7100 - accuracy: 0.5345
----------- Starting trial: run-58-----------
{'num_layers': 5, 'num_nodes': 8, 'optimizer': 'sgd', 'batch_size': 210, 'max_epochs': 25, 'l2 regularizer': 0.001}
Fold nr 2
Epoch 1/25
47/47 - 0s - loss: 0.7191 - accuracy: 0.5294 - val_loss: 0.7209 - val_accuracy: 0.5252
Epoch 2/25
47/47 - 0s - loss: 0.7161 - accuracy: 0.5600 - val_loss: 0.7180 - val_accuracy: 0.5482
Epoch 3/25
47/47 - 0s - los

96/96 [==============================] - ETA: 0s - loss: 0.7237 - accuracy: 0.62 - ETA: 0s - loss: 0.7244 - accuracy: 0.46 - ETA: 0s - loss: 0.7245 - accuracy: 0.46 - 0s 2ms/step - loss: 0.7244 - accuracy: 0.4711
----------- Starting trial: run-59-----------
{'num_layers': 5, 'num_nodes': 8, 'optimizer': 'sgd', 'batch_size': 210, 'max_epochs': 25, 'l2 regularizer': 0.01}
Fold nr 3
Epoch 1/25
47/47 - 0s - loss: 0.7198 - accuracy: 0.5331 - val_loss: 0.7198 - val_accuracy: 0.5421
Epoch 2/25
47/47 - 0s - loss: 0.7159 - accuracy: 0.5444 - val_loss: 0.7165 - val_accuracy: 0.5466
Epoch 3/25
47/47 - 0s - loss: 0.7129 - accuracy: 0.5548 - val_loss: 0.7135 - val_accuracy: 0.5544
Epoch 4/25
47/47 - 0s - loss: 0.7097 - accuracy: 0.5671 - val_loss: 0.7101 - val_accuracy: 0.5732
Epoch 5/25
47/47 - 0s - loss: 0.7063 - accuracy: 0.5775 - val_loss: 0.7066 - val_accuracy: 0.5827
Epoch 6/25
47/47 - 0s - loss: 0.7026 - accuracy: 0.5873 - val_loss: 0.7028 - val_accuracy: 0.5929
Epoch 7/25
47/47 - 0s - loss

33/33 - 0s - loss: 0.7428 - accuracy: 0.4874 - val_loss: 0.7351 - val_accuracy: 0.4735
Epoch 2/12
33/33 - 0s - loss: 0.7317 - accuracy: 0.4640 - val_loss: 0.7271 - val_accuracy: 0.4707
Epoch 3/12
33/33 - 0s - loss: 0.7246 - accuracy: 0.4927 - val_loss: 0.7212 - val_accuracy: 0.5129
Epoch 4/12
33/33 - 0s - loss: 0.7191 - accuracy: 0.5538 - val_loss: 0.7163 - val_accuracy: 0.5851
Epoch 5/12
33/33 - 0s - loss: 0.7145 - accuracy: 0.6007 - val_loss: 0.7120 - val_accuracy: 0.6118
Epoch 6/12
33/33 - 0s - loss: 0.7105 - accuracy: 0.6168 - val_loss: 0.7081 - val_accuracy: 0.6258
Epoch 7/12
33/33 - 0s - loss: 0.7068 - accuracy: 0.6281 - val_loss: 0.7044 - val_accuracy: 0.6241
96/96 [==============================] - ETA: 0s - loss: 0.7038 - accuracy: 0.63 - 0s 883us/step - loss: 0.7058 - accuracy: 0.6326
----------- Starting trial: run-60-----------
{'num_layers': 5, 'num_nodes': 8, 'optimizer': 'sgd', 'batch_size': 300, 'max_epochs': 12, 'l2 regularizer': 0.001}
Fold nr 5
Epoch 1/12
33/33 - 0s 

Epoch 2/12
33/33 - 0s - loss: 0.7143 - accuracy: 0.5605 - val_loss: 0.7133 - val_accuracy: 0.5599
Epoch 3/12
33/33 - 0s - loss: 0.7113 - accuracy: 0.5709 - val_loss: 0.7109 - val_accuracy: 0.5628
Epoch 4/12
33/33 - 0s - loss: 0.7091 - accuracy: 0.5733 - val_loss: 0.7092 - val_accuracy: 0.5657
Epoch 5/12
33/33 - 0s - loss: 0.7074 - accuracy: 0.5784 - val_loss: 0.7076 - val_accuracy: 0.5665
Epoch 6/12
33/33 - 0s - loss: 0.7058 - accuracy: 0.5846 - val_loss: 0.7061 - val_accuracy: 0.5694
Epoch 7/12
33/33 - 0s - loss: 0.7041 - accuracy: 0.5877 - val_loss: 0.7046 - val_accuracy: 0.5755
96/96 [==============================] - ETA: 0s - loss: 0.7007 - accuracy: 0.50 - ETA: 0s - loss: 0.7039 - accuracy: 0.58 - 0s 808us/step - loss: 0.7033 - accuracy: 0.5942
Combined accuray:  0.594615888595581
----------- Starting trial: run-62-----------
{'num_layers': 5, 'num_nodes': 8, 'optimizer': 'sgd', 'batch_size': 300, 'max_epochs': 25, 'l2 regularizer': 0.001}
Fold nr 1
Epoch 1/25
33/33 - 0s - loss: 

33/33 - 1s - loss: 0.7309 - accuracy: 0.4948 - val_loss: 0.7281 - val_accuracy: 0.4912
Epoch 2/25
33/33 - 0s - loss: 0.7205 - accuracy: 0.5073 - val_loss: 0.7185 - val_accuracy: 0.5146
Epoch 3/25
33/33 - 0s - loss: 0.7147 - accuracy: 0.5209 - val_loss: 0.7135 - val_accuracy: 0.5384
Epoch 4/25
33/33 - 0s - loss: 0.7120 - accuracy: 0.5330 - val_loss: 0.7110 - val_accuracy: 0.5568
Epoch 5/25
33/33 - 0s - loss: 0.7104 - accuracy: 0.5423 - val_loss: 0.7092 - val_accuracy: 0.5593
Epoch 6/25
33/33 - 0s - loss: 0.7092 - accuracy: 0.5481 - val_loss: 0.7079 - val_accuracy: 0.5650
Epoch 7/25
33/33 - 0s - loss: 0.7082 - accuracy: 0.5559 - val_loss: 0.7067 - val_accuracy: 0.5696
96/96 [==============================] - ETA: 0s - loss: 0.6596 - accuracy: 0.46 - ETA: 0s - loss: 0.7050 - accuracy: 0.57 - 0s 801us/step - loss: 0.7068 - accuracy: 0.5614
----------- Starting trial: run-63-----------
{'num_layers': 5, 'num_nodes': 8, 'optimizer': 'sgd', 'batch_size': 300, 'max_epochs': 25, 'l2 regularizer

In [ ]:
%load_ext tensorboard 
